In [15]:
import torch
_ = torch.tensor([0.2126, 0.7152, 0.0722], names=['c'])

In [16]:
img_t = torch.randn(3,5,5) # (通道数，行数，列数)
weights = torch.tensor([0.2126, 0.7152, 0.0722])

In [17]:
batch_t = torch.randn(2,3,5,5) # (批量大小，通道数，行数，列数)

RGB 通道有时在第 0 维，有时在第一维，但我们可以通过从末端开始计数来归纳：它们总是在从末端开始的第三维中。因此，惰性的未加权平均值可以写成如下的形式：

In [18]:
# 对倒数第三个维度求平均值，这样就能把三通道彩色图转换为灰度图
img_gray_naive = img_t.mean(-3)
batch_gray_naive = batch_t.mean(-3)
img_gray_naive.shape, batch_gray_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

PyTorch 将允许我们对相同形状的张量进行乘法运算，也允许与给定维度中其中一个操作数大小为 1 的张量进行运算。它还会自动附加大小为 1 的前导维度，这个特性被称为广播。形状为(2,3,5,5)的 batch_t 乘一个形状为(3,1,1) unsqueezed_weights 张量（在乘法操作中，unsqueezed_weights 的第二和第三维度（尺寸为1的维度）被扩展（或广播）以匹配 batch_t 的第三和第四维度），得到一个形状为(2,3,5,5)的张量，由此我们可以从末端开始对第三维（ 3 个通道）进行求和：

In [24]:
# 在 weights 张量的最后一个维度后添加两个新的维度。这通常用于将一个一维或二维的张量转换为三维或四维，以便进行某些特定的张量操作，例如卷积。
# unsqueeze 不会修改原始张量，而是返回一个新的张量；而 unsqueeze_ 是一个就地操作（in-place operation），它会直接修改原始张量。
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)

img_weights = (img_t * unsqueezed_weights)
batch_weights = (batch_t * unsqueezed_weights)

img_gray_weighted = img_weights.sum(-3)
batch_gray_naive = batch_weights.sum(-3)

unsqueezed_weights.shape, img_t.shape, img_weights.shape, batch_t.shape, batch_weights.shape, img_gray_weighted.shape, batch_gray_naive.shape

(torch.Size([3, 1, 1]),
 torch.Size([3, 5, 5]),
 torch.Size([3, 5, 5]),
 torch.Size([2, 3, 5, 5]),
 torch.Size([2, 3, 5, 5]),
 torch.Size([5, 5]),
 torch.Size([2, 5, 5]))

In [29]:
weights_named = torch.tensor([0.2126, 0.7152, 0.0722], names=['channels'])
weights_named.shape, weights_named.names

(torch.Size([3]), ('channels',))

当我们已经有一个张量并且想要为其添加名称但不改变现有的名称时，我们可以对其调用refine_names()方法。与索引类似，省略号（ …）允许你省略任意数量的维度。使用 rename()兄弟方法，还可以覆盖或删除（通过传入 None）现有名称：

In [28]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print("img named:", img_named.shape, img_named.names)
print("batch named:", batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


对于有 2 个输入的操作，除了常规维度检查，即检查张量维度是否相同，以及是否一个张量维度为 1 并且可以广播给另一个张量， PyTorch 还将检查张量名称。到目前为止，它还没有提供自动维度对齐功能，因此我们需要显式地进行此操作。 align_as()方法返回一个张量，其中添加了缺失的维度，现有的维度按正确的顺序排列如下：

In [30]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

接收维度参数的函数，例如 sum()，也接收命名维度：

In [31]:
gray_named = (img_named * weights_aligned).sum('channels')
gray_named.shape, gray_named.names

(torch.Size([5, 5]), ('rows', 'columns'))